In [24]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.WiringDiagrams
using Catlab.Programs
using Catlab.Graphics
using Catlab.Graphics: Graphviz

using AlgebraicDynamics
using AlgebraicDynamics.DWDDynam
using AlgebraicDynamics.UWDDynam
using AlgebraicDynamics.CPortGraphDynam
using AlgebraicDynamics.CPortGraphDynam: grid

using LabelledArrays

using DynamicalSystems, OrdinaryDiffEq
using Plots
plotly()

Plots.PlotlyBackend()

In [25]:
# Pick a composition syntax

W = 10; H = 1  # we'll create a grid with width 10 and height 1

# initialize the instance
stencil = OpenCPortGraph()

# add the boxes
boxes = reshape(add_parts!(stencil, :Box, W*H), H, W) 

ports = map(i -> add_parts!(stencil, :Port, 4,  box = i), boxes)

# add up/down wires
map(view(boxes, 1:(H - 1), 1:W)) do b
    add_part!(stencil, :Wire, src = ports[b][3], tgt = ports[b + 1][1])
    add_part!(stencil, :Wire, tgt = ports[b][3], src = ports[b + 1][1])
end

# add left/right wires
map(view(boxes, 1:H, 1:(W-1))) do b
    add_part!(stencil, :Wire, src = ports[b][2], tgt = ports[b+H][4])
    add_part!(stencil, :Wire, tgt = ports[b][2], src = ports[b+H][4])
end

# add outer ports
add_parts!(stencil, :OuterPort, W, con = map(i -> ports[1,i][1], 1:W))
add_parts!(stencil, :OuterPort, W, con = map(i -> ports[H,i][3], 1:W))
add_parts!(stencil, :OuterPort, H, con = map(i -> ports[i,1][4], 1:H))
add_parts!(stencil, :OuterPort, H, con = map(i -> ports[i,W][2], 1:H))

stencil

Port,box
1,1
2,1
3,1
4,1
5,2
6,2
7,2
8,2
9,3
10,3


In [49]:
# Choose a primitive model. We will fill each box with the same primitive model
# We are attempting to encode the elastic curvature of a beam when presented with a load on the free end
# Via trial and error the order of ports appears to be as follows
#
#                 1
#                 |
#            2---------4
#                 |
#                 3

point_model = DiscreteMachine{Float64}(
    4, # number of inputs, (x(left, right, top, bottom))
    1, # number of states, x(i)
    4, # number of outputs
    # dynamics: u - the state vector (current node), x - the input vector (neighbors)
    (u,x,p,t) -> [-1*(x[1])*(((p.γ*((x[4])^2)*(3*p.L-x[4])) - 
                (p.γ*((u[1])^2)*(3*p.L-u[1]))) /
                (x[4]-u[1]))], # displacement, u(x,y)
    u -> repeat(u, 4) # the read out function, for each machine 
)


DiscreteMachine(ℝ^1 × ℝ^4 → ℝ^1)

In [33]:
# Equations I want to encode:
# v(i) = Px^2/6EI (3L-x)
# θ(i) = dv/dx
# u(i) = -y * θ(i) 
# ϵ(i) = -y(i)* du/dx
# sigma(x) = ϵ(i) * E


In [50]:
# Compose!
# Apply the point model to each box of the stencil
displacement_eq = oapply(stencil, point_model)

DiscreteMachine(ℝ^10 × ℝ^22 → ℝ^10)

In [68]:
# Solve and plot!

# initial coundition
a = Any[1., 2., 3., 4., 5., 6., 7., 8. ,9., 10.]
u0 = convert(Array{Float64,1}, a) 


# boundary condition
xs = Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
xs = convert(Array{Float64,1}, xs)


# parameters
P = 9e6; # load, N
E = 5e6 # Young's modulus, N/m^2
I = 1/12 # second moment of area, m^4

length = 10 # beam length, m
delta_t = 1
gamma = P/(6*E*I) # just used to collect all the constants in one variable aside from length

params = LVector(γ = gamma, L = length)
nsteps = 25

# find the trajectory
traj = trajectory(displacement_eq, u0, xs, params, nsteps)


10-dimensional Dataset{Float64} with 26 points
     1.0             2.0          …      9.0             10.0
  -104.4          -298.8             -1054.8          -1076.4
 50512.9             5.16497e5           1.17539e7        1.2494e7
     9.18013e9       1.06342e12          1.3469e15        1.58798e15
     3.0339e20       4.10654e24          1.54478e31       2.33089e31
     3.31363e41      6.07136e49   …      1.57862e63       4.11124e63
     3.95284e83      1.32701e100         1.18795e127      9.31838e127
     5.62499e167     6.33947e200       NaN              NaN
    Inf            NaN                 NaN              NaN
   NaN             NaN                 NaN              NaN
   NaN             NaN            …    NaN              NaN
   NaN             NaN                 NaN              NaN
   NaN             NaN                 NaN              NaN
   NaN             NaN                 NaN              NaN
   NaN             NaN                 NaN              NaN
   Na

In [69]:
plot(traj[:2], label = "first timestep", linewidth=3)

xlabel!("longitudinal beam axis")
ylabel!("beam displacement")

<!DOCTYPE html>
 
 
 Plots.jl

In [70]:
plot(traj[:3], label = "second timestep", linewidth=3)

xlabel!("longitudinal beam axis")
ylabel!("beam displacement")

<!DOCTYPE html>
 
 
 Plots.jl